In [1]:
# Get datafiles in the input directory. Create a sorted list of dates
# when the snapshots where downloaded.

import os
import re

archivedir = '../data/nyc-open-data/y43c-5n92/archive'
datadir = '../data/nyc-open-data/y43c-5n92/data'

datafiles = dict()
for filename in os.listdir(datadir):
    if re.match(r'y43c-5n92.\w{8}.tsv.gz', filename):
        date = filename.split('.')[1]
        datafiles[date] = os.path.join(datadir, filename)

        
downloads = sorted(list(datafiles.keys()))

In [2]:
import pandas as pd
df = pd.read_csv(datafiles[downloads[0]], compression='gzip', delimiter='\t')

In [3]:
df.columns

Index(['Site', 'Date', 'Turbidity(NTU) at 12AM', 'Turbidity(NTU) at 4AM',
       'Turbidity(NTU) at 8AM', 'Turbidity(NTU) at 12PM',
       'Turbidity(NTU) at 4PM', 'Turbidity(NTU) at 8PM',
       'Average 24hrTurbidity(NTU)', 'Coliform, Fecal(fc/100mL)'],
      dtype='object')

In [4]:
df.head()

,Site,Date,Turbidity(NTU) at 12AM,Turbidity(NTU) at 4AM,Turbidity(NTU) at 8AM,Turbidity(NTU) at 12PM,Turbidity(NTU) at 4PM,Turbidity(NTU) at 8PM,Average 24hrTurbidity(NTU),"Coliform, Fecal(fc/100mL)"
0,DEL18DT,08/31/2019,0.90,0.85,0.85,0.75,0.75,0.70,0.80,E1
1,DEL18DT,08/30/2019,0.80,0.80,0.90,0.90,1.00,0.95,0.89,<1
2,DEL18DT,08/29/2019,0.85,0.85,0.85,0.80,0.80,0.85,0.83,E4
3,DEL18DT,08/28/2019,0.95,0.95,0.80,0.85,0.75,0.85,0.86,<1
4,DEL18DT,08/27/2019,0.80,0.95,0.95,0.85,0.80,0.85,0.87,<1


In [5]:
df.shape

(1793, 10)

In [6]:
df = pd.read_csv(datafiles[downloads[1]], compression='gzip', delimiter='\t')
df.head()

,Site,Date,Turbidity(NTU) at 12AM,Turbidity(NTU) at 4AM,Turbidity(NTU) at 8AM,Turbidity(NTU) at 12PM,Turbidity(NTU) at 4PM,Turbidity(NTU) at 8PM,Average 24hrTurbidity(NTU),"Coliform, Fecal(fc/100mL)"
0,DEL18DT,09/30/2019,0.50,0.55,0.55,0.45,0.55,0.55,0.53,E1
1,DEL18DT,09/29/2019,0.50,0.60,0.55,0.55,0.55,0.55,0.55,<1
2,DEL18DT,09/28/2019,0.60,0.55,0.55,0.70,0.65,0.55,0.60,E2
3,DEL18DT,09/27/2019,0.60,0.60,0.60,0.55,0.55,0.55,0.58,<1
4,DEL18DT,09/26/2019,0.55,0.55,0.60,0.60,0.70,0.60,0.60,<1


In [7]:
df.shape

(1823, 10)

In [8]:
import histore as hs

# Merge all dataset snapshots in the directory into a persistent archive
# Ensure to set the replace flag to True. Otherwise, the snapshots are
# appended to an existing archive everytime the cell is executed.

archive = hs.PersistentArchive(basedir=archivedir, replace=True, primary_key=['Site', 'Date'])
for date in downloads:
    df = pd.read_csv(datafiles[date], compression='gzip', delimiter='\t')
    s = archive.commit(df, description=date)
    print('Snapshot {} ({})'.format(s.version, s.description))
    print('------------\n')
    archive.diff(s.version - 1, s.version).describe()
    print('\n\n')

Snapshot 0 (20190927)
------------

Schema Changes
Inserted Columns : 10
Deleted Columns  : 0
Moved Columns    : 0
Renamed Columns  : 0

Data Changes
Inserted Rows    : 1793
Deleted Rows     : 0
Moved Rows       : 0
Updated Rows     : 0
Updated Values   : 0



Snapshot 1 (20191004)
------------

Schema Changes
Inserted Columns : 0
Deleted Columns  : 0
Moved Columns    : 0
Renamed Columns  : 0

Data Changes
Inserted Rows    : 30
Deleted Rows     : 0
Moved Rows       : 1793
Updated Rows     : 8
Updated Values   : 22



Snapshot 2 (20191104)
------------

Schema Changes
Inserted Columns : 0
Deleted Columns  : 0
Moved Columns    : 0
Renamed Columns  : 0

Data Changes
Inserted Rows    : 30
Deleted Rows     : 0
Moved Rows       : 1822
Updated Rows     : 12
Updated Values   : 24



Snapshot 3 (20191111)
------------

Schema Changes
Inserted Columns : 0
Deleted Columns  : 0
Moved Columns    : 0
Renamed Columns  : 0

Data Changes
Inserted Rows    : 31
Deleted Rows     : 0
Moved Rows       : 0
U

In [9]:
# Which rows/values where updated in the second snapshot (compared to the first snapshot)

for upd_row in archive.diff(0, 1).rows().update():
    if upd_row.updated_cells():
        for colid, upd in upd_row.updated_cells().items():
            colname = archive.schema().columns[colid].name.at_version(0)
            print("UPDATE '{}' FROM {} FOR ROW {}".format(colname, upd, upd_row.key))

UPDATE 'Turbidity(NTU) at 12PM' FROM None TO 0.75 FOR ROW ('DEL18DT', '05/03/2019')
UPDATE 'Turbidity(NTU) at 4PM' FROM None TO 0.65 FOR ROW ('DEL18DT', '05/03/2019')
UPDATE 'Turbidity(NTU) at 8PM' FROM None TO 0.8 FOR ROW ('DEL18DT', '05/03/2019')
UPDATE 'Coliform, Fecal(fc/100mL)' FROM None TO <1 FOR ROW ('DEL18DT', '05/03/2019')
UPDATE 'Turbidity(NTU) at 12PM' FROM 0.75 TO None FOR ROW ('DEL18DT', '05/03/2019')
UPDATE 'Turbidity(NTU) at 4PM' FROM 0.65 TO None FOR ROW ('DEL18DT', '05/03/2019')
UPDATE 'Turbidity(NTU) at 8PM' FROM 0.8 TO None FOR ROW ('DEL18DT', '05/03/2019')
UPDATE 'Coliform, Fecal(fc/100mL)' FROM <1 TO None FOR ROW ('DEL18DT', '05/03/2019')
UPDATE 'Turbidity(NTU) at 12AM' FROM 0.8 TO None FOR ROW ('DEL18DT', '06/14/2015')
UPDATE 'Turbidity(NTU) at 12AM' FROM None TO 0.8 FOR ROW ('DEL18DT', '06/14/2015')
UPDATE 'Turbidity(NTU) at 12PM' FROM 0.65 TO None FOR ROW ('DEL18DT', '10/03/2017')
UPDATE 'Turbidity(NTU) at 4PM' FROM 0.7 TO None FOR ROW ('DEL18DT', '10/03/2017')


In [10]:
# Explain changes in row ('DEL18DT', '12/13/2015') for
# column 'Turbidity(NTU) at 12AM'

df = pd.read_csv(datafiles[downloads[0]], compression='gzip', delimiter='\t')
df1 = df[df.Site.eq('DEL18DT') & df.Date.eq('12/13/2015')]
print(df1[['Site', 'Date', 'Turbidity(NTU) at 12AM']])

         Site        Date  Turbidity(NTU) at 12AM
1558  DEL18DT  12/13/2015                     NaN
1559  DEL18DT  12/13/2015                     0.7


In [11]:
df = pd.read_csv(datafiles[downloads[1]], compression='gzip', delimiter='\t')
df2 = df[df.Site.eq('DEL18DT') & df.Date.eq('12/13/2015')]
print(df2[['Site', 'Date', 'Turbidity(NTU) at 12AM']])

         Site        Date  Turbidity(NTU) at 12AM
1588  DEL18DT  12/13/2015                     0.7
1589  DEL18DT  12/13/2015                     NaN
